In [1]:
%pip install -U weaviate-client
%pip install -U tqdm
%pip install -U ipywidgets
!jupyter nbextension enable --py widgetsnbextension

  Using cached weaviate_client-4.9.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached validators-0.34.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached Authlib-1.3.1-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached grpcio-1.68.1-cp313-cp313-macosx_10_13_universal2.whl.metadata (3.9 kB)
  Using cached grpcio_tools-1.68.1-cp313-cp313-macosx_10_13_universal2.whl.metadata (5.3 kB)
  Using cached grpcio_health_checking-1.68.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached cryptography-44.0.0-cp39-abi3-macosx_10_9_universal2.whl.metadata (5.7 kB)
  Using cached protobuf-5.29.1-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached setuptools-75.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcor

In [2]:
import weaviate.classes as wvc
import weaviate
from weaviate.classes.config import Configure, Property, DataType

with weaviate.connect_to_local(host="localhost", port=8080, grpc_port=50051) as client:
    ollama_api_endpoint = "http://host.docker.internal:11434"
    
    if client.collections.exists("trivia"):
        client.collections.delete("trivia")
    
    client.collections.create(
        name="trivia",
        description="Open trivia database dump",
        replication_config=Configure.replication(factor=1),
        sharding_config=Configure.sharding(virtual_per_physical=128, desired_count=1),
        vectorizer_config=[
            wvc.config.Configure.NamedVectors.text2vec_ollama(name="nomic",
                                                              api_endpoint=ollama_api_endpoint,
                                                              model="nomic-embed-text")
        ],
        generative_config=wvc.config.Configure.Generative.ollama(api_endpoint=ollama_api_endpoint, model="llama3.2"),
        properties=[
            Property(name="type", data_type=DataType.TEXT),
            Property(name="category", data_type=DataType.TEXT),
            Property(name="question", data_type=DataType.TEXT),
            Property(name="correct_answer", data_type=DataType.TEXT),
            Property(name="incorrect_answes", data_type=DataType.TEXT_ARRAY),
            Property(name="difficulty", data_type=DataType.TEXT, skip_vectorization=True)
        ]
    )
    




/Users/gaspar_d/Projects/weaviate-playbooks/.venv/lib/python3.13/site-packages/weaviate/warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 2.5.1.dev2545+ge2799c1. The latest version is 4.9.6.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [ ]:
import weaviate.classes as wvc
import weaviate
import requests
import time
from tqdm.notebook import tqdm

with weaviate.connect_to_local(host="localhost", port=8080, grpc_port=50051) as client:
    with client.batch.dynamic() as batch:
      for rec in tqdm(range(500)):
        if batch.number_errors > 10:
          break
        results = requests.get(url="https://opentdb.com/api.php?amount=25").json()
        for trivia in results["results"]:
          batch.add_object(
            collection="trivia",
            uuid=weaviate.util.generate_uuid5(trivia["question"]),
            properties={
              "type": trivia["type"],
              "category": trivia["category"],
              "question": trivia["question"],
              "correct_answer": trivia["correct_answer"],
              "incorrect_answers": trivia["incorrect_answers"],
              "difficulty": trivia["difficulty"],
            }
          )
        time.sleep(5)
    print(client.batch.failed_objects)
    

  0%|          | 0/400 [00:00<?, ?it/s]

[]
